In [83]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16, DenseNet201, ResNet50, InceptionV3, InceptionResNetV2
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from sklearn.metrics import classification_report, accuracy_score

In [84]:
folder = '/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/20180501/ProcessedImages/'
unlabeled_names = [f for f in os.listdir('/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/ProcessedImages/unlabeled') if 'jpg' in f]
unknown2_names = os.listdir('/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/ProcessedImages/test/unknown2/') 


In [92]:
subfolders = [d for d in os.listdir(folder) if os.path.isdir(folder + d) and not d.isalpha()]
count = 0
to_Remove = []
for i in unknown2_names:
    for j in unlabeled_names:
        if i == j:
            count += 1
            to_Remove.append(i)
            break
len(unlabeled_names)

3312

In [93]:

new_folder = folder + 'unknown/'
if(not os.path.exists(new_folder)):
    os.mkdir(new_folder)
remaining = pd.DataFrame()
for d in subfolders:
    df = pd.read_csv(folder + d + "/" + d + ".csv").set_index('filename')
    for f in [file for file in os.listdir(folder + d) if('jpg' in file and file in unlabeled_names)]:
        copyfile(folder + d + '/' + f, new_folder + f)
        


In [94]:
remaining['filename'] = os.listdir(new_folder)

In [95]:
remaining.head()

,filename
0,125355_forcadinho-np3_frame199.jpg
1,113610_cam_survey_1_frame137.jpg
2,113610_cam_survey_1_frame323.jpg
3,125355_forcadinho-np3_frame396.jpg
4,132143_forcadinho-np3_frame2900.jpg


In [97]:
remaining.to_csv(folder + 'unknown.csv', index=False)
#remaining = pd.read_csv(folder + 'unknown.csv')
remaining.head()

,filename
0,125355_forcadinho-np3_frame199.jpg
1,113610_cam_survey_1_frame137.jpg
2,113610_cam_survey_1_frame323.jpg
3,125355_forcadinho-np3_frame396.jpg
4,132143_forcadinho-np3_frame2900.jpg


# Predictions


In [98]:
test_path = folder + 'test/'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        shuffle=False)

Found 3312 images belonging to 1 classes.


In [99]:
def loadModel(model):
    if('dense' in model):
        print('densenet')
        return DenseNet201()
    elif('res' in model):
        print('resnet')
        return ResNet50()
    elif('inceptionv3' in model):
        print('inceptionv3')
        return InceptionV3()
    else:
        print("vgg16")
        return VGG16(weights='imagenet')
        

In [100]:
weights = 'densenet.h5'
model = loadModel(weights)
x = Dense(5, activation='softmax', name='predictions')(model.layers[-2].output)
M = Model(input=model.input, output=x)

densenet


/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  after removing the cwd from sys.path.


In [101]:
optimizer = SGD(lr=0.001)
M.load_weights('/home/diegues/git/AutomaticHabitatMapping/src/best_weights/' + weights)
M.compile(loss='categorical_crossentropy',
                optimizer=optimizer, 
                metrics=['acc'])

In [102]:
predictions_densenet = pd.DataFrame(M.predict_generator(test_generator, steps = len(remaining), verbose=1))
predictions_densenet.columns = ['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']
predictions_densenet['level3'] = predictions_densenet.idxmax(axis=1)

3312/3312 [==============================] - 183s 55ms/step


In [103]:
remaining['densenet'] = predictions_densenet['level3']
remaining.to_csv(folder + 'unknown.csv', index=False)

In [104]:
weights = 'resnet.h5'
model = loadModel(weights)
x = Dense(5, activation='softmax', name='predictions')(model.layers[-2].output)
M = Model(input=model.input, output=x)
optimizer = SGD(lr=0.001)
M.load_weights('/home/diegues/git/AutomaticHabitatMapping/src/best_weights/' + weights)
M.compile(loss='categorical_crossentropy',
                optimizer=optimizer, 
                metrics=['acc'])

resnet


/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  after removing the cwd from sys.path.


In [105]:
predictions_resnet = pd.DataFrame(M.predict_generator(test_generator, steps = len(remaining), verbose=1))
predictions_resnet.columns = ['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']
predictions_resnet['level3'] = predictions_resnet.idxmax(axis=1)

3312/3312 [==============================] - 87s 26ms/step


In [106]:
remaining['resnet'] = predictions_resnet['level3']
remaining.to_csv(folder + 'unknown.csv', index=False)

In [107]:
weights = 'vgg16_0_001_full_training.h5'
model = loadModel(weights)
fc1 = model.layers[-3]
fc2 = model.layers[-2]
predictions = Dense(5, activation='softmax', name='predictions')
dropout1 = Dropout(0.5)
dropout2 = Dropout(0.5)
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)
M = Model(input=model.input, output=predictors)

optimizer = SGD(lr=0.001)
M.load_weights('/home/diegues/git/AutomaticHabitatMapping/src/best_weights/' + weights)
M.compile(loss='categorical_crossentropy',
                optimizer=optimizer, 
                metrics=['acc'])

vgg16


/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  if sys.path[0] == '':


In [108]:
predictions_vgg = pd.DataFrame(M.predict_generator(test_generator, steps = len(remaining), verbose=1))
predictions_vgg.columns = ['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']
predictions_vgg['level3'] = predictions_vgg.idxmax(axis=1)

3312/3312 [==============================] - 94s 28ms/step


In [109]:
remaining['vgg'] = predictions_vgg['level3']
remaining.to_csv(folder + 'unknown.csv', index=False)

In [110]:
remaining.head()

,filename,densenet,resnet,vgg
0,125355_forcadinho-np3_frame199.jpg,A3.1-A4.1,A3.7-A4.7,A3.1-A4.1
1,113610_cam_survey_1_frame137.jpg,A3.1-A4.1,A3.1-A4.1,A3.7-A4.7
2,113610_cam_survey_1_frame323.jpg,A3.1-A4.1,A3.7-A4.7,A3.1-A4.1
3,125355_forcadinho-np3_frame396.jpg,A3.7-A4.7,A5.2,A3.7-A4.7
4,132143_forcadinho-np3_frame2900.jpg,A3.7-A4.7,A5.2,A3.7-A4.7


In [111]:
remaining.densenet.value_counts()

A3.1-A4.1    1953
A3.7-A4.7     682
A5.1          546
A5.2          103
A5.4           28
Name: densenet, dtype: int64

In [112]:
remaining.resnet.value_counts()

A3.1-A4.1    1754
A3.7-A4.7     839
A5.1          471
A5.2          217
A5.4           31
Name: resnet, dtype: int64

In [113]:
remaining.vgg.value_counts()

A3.1-A4.1    2172
A3.7-A4.7     541
A5.1          483
A5.2           77
A5.4           39
Name: vgg, dtype: int64